<a href="https://colab.research.google.com/github/andrnsgnmez/DATASET-HERRAMIENTAS-DEEP-LEARNING/blob/main/Proyecto_1_Vision_computacional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! git clone https://github.com/andrnsgnmez/DATASET-HERRAMIENTAS-DEED-LEARNING.git

Cloning into 'DATASET-HERRAMIENTAS-DEED-LEARNING'...
remote: Enumerating objects: 314, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 314 (delta 0), reused 0 (delta 0), pack-reused 311 (from 1)
Receiving objects: 100% (314/314), 864.40 MiB | 34.01 MiB/s, done.
Resolving deltas: 100% (6/6), done.
Updating files: 100% (283/283), done.


In [ ]:
!pip install tensorflow==2.15

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 72.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 113.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.9 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.16.0
    Uninstalling wrapt-1.16.0:
      Successfully uninstalled wrapt-1.16.0
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorboard
    Found existing installatio

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay , classification_report
from PIL import Image
import os
import cv2

In [ ]:
tf.__version__

In [ ]:
train_path = '/content/DATASET-HERRAMIENTAS-DEED-LEARNING/DATASET/train'
test_path = '/content/DATASET-HERRAMIENTAS-DEED-LEARNING/DATASET/test'

In [ ]:
# Rutas a las carpetas de imágenes
train_path = '/content/DATASET-HERRAMIENTAS-DEED-LEARNING/DATASET/train'
test_path = '/content/DATASET-HERRAMIENTAS-DEED-LEARNING/DATASET/test'

# Función para obtener tamaños de imágenes en una carpeta
def get_image_sizes(folder_path):
    image_sizes = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            # Verifica si el archivo es una imagen
            if file.endswith(('jpg', 'jpeg', 'png')):
                image_path = os.path.join(root, file)
                with Image.open(image_path) as img:
                    width, height = img.size
                    # Incluye el nombre de la subcarpeta (clase) para mayor claridad
                    class_name = os.path.basename(root)
                    image_sizes.append((class_name, file, width, height))
                    print(f"Clase: {class_name} - Imagen: {file} - Ancho: {width}, Alto: {height}")
    return image_sizes

# Obtener tamaños de imágenes en las carpetas de entrenamiento y prueba
train_image_sizes = get_image_sizes(train_path)
test_image_sizes = get_image_sizes(test_path)

# Opcional: imprime el tamaño más común o estadística general para el conjunto de entrenamiento
if train_image_sizes:
    unique_train_sizes = set((class_name, width, height) for class_name, _, width, height in train_image_sizes)
    print("\nTamaños únicos de las imágenes en el dataset de entrenamiento:")
    for size in unique_train_sizes:
        print(f"Clase: {size[0]} - Tamaño: {size[1]}x{size[2]}")
else:
    print("No se encontraron imágenes en el dataset de entrenamiento.")

# Opcional: imprime el tamaño más común o estadística general para el conjunto de prueba
if test_image_sizes:
    unique_test_sizes = set((class_name, width, height) for class_name, _, width, height in test_image_sizes)
    print("\nTamaños únicos de las imágenes en el dataset de prueba:")
    for size in unique_test_sizes:
        print(f"Clase: {size[0]} - Tamaño: {size[1]}x{size[2]}")
else:
    print("No se encontraron imágenes en el dataset de prueba.")



In [ ]:
# parametros
img_height , img_width = 224 , 224
batch_size = 32

train_datagen = ImageDataGenerator(rescale = 1.0/255 , rotation_range = 20 , width_shift_range = 0.2 , height_shift_range = 0.2 , shear_range = 0.2 , zoom_range = 0.2 , horizontal_flip = True , fill_mode = 'nearest')
validation_datagen = ImageDataGenerator(rescale = 1.0/255)

train_generator = train_datagen.flow_from_directory(train_path , target_size = (img_height,img_width), batch_size = batch_size , class_mode ='categorical')
validation_generator = validation_datagen.flow_from_directory(test_path , target_size = (img_height,img_width), batch_size = batch_size , class_mode ='categorical')

In [ ]:
model = Sequential([Input(shape=(img_height, img_width, 3)),
        Conv2D(16, (3,3), activation = 'relu'), MaxPooling2D(pool_size=(2,2)),
        Conv2D(32,(3,3), activation='relu'), MaxPooling2D(pool_size=(2,2)),
        Conv2D(64,(3,3), activation='relu'), MaxPooling2D(pool_size=(2,2)),
        Conv2D(128,(3,3), activation='relu'), MaxPooling2D(pool_size=(2,2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(5, activation = 'softmax')])

In [ ]:
model.summary()
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
model.compile(loss = 'categorical_crossentropy',  optimizer = 'adam', metrics = ['accuracy'])